# Regression

## Task 8: Loading and Initial Data Preparation

Prepare the California housing dataset for further regression analysis.  
In this step, we load the data, analyze its structure, and identify the unique values of a categorical variable.


In [ ]:
# Imports for this notebook
import pandas as pd


In [ ]:
# a) Load the dataset from the CSV file
df = pd.read_csv("housing.csv")
df.head()


In [ ]:
# b) Display basic information about the dataset
df.info()

df.describe()


In [ ]:
# c) Unique values of the "ocean_proximity" column
print("Unique values in 'ocean_proximity':")
df["ocean_proximity"].value_counts()
